In [45]:
library(tidyverse)

results_df <- read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", show_col_types = FALSE)

player_ssns <- read_csv("../output/player_ssns.csv", show_col_types = FALSE)

In [46]:
files <- Sys.glob("./csv/*.csv")

long_ssns <- data.frame()

for (file in files) {
    season = str_remove_all(file, "[\\.\\/csv\\/]")
    season = str_replace_all(season, "-", "/")
    
    df <- read_csv(file, show_col_types = FALSE) %>%
        mutate(
            season = season,
            game_no = row_number()
        )

    ssn_df <- results_df %>%
    select(
        season,
        ssn_game_no,
        game_date,
        competition,
        venue,
        opposition,
        outcome,
        score,
        attendance
    ) %>%
    rename(
        game_no = ssn_game_no
    ) %>%
    inner_join(
        df,
        by = c("season", "game_no")
    ) %>%
    arrange(
        game_date
    ) %>%
        mutate(
        across(-game_no, as.character)
    ) %>%
    select(
        -game_date,
        -venue,
        -opposition,
        -outcome,
        -score,
        -attendance
    ) %>%
    pivot_longer(
        cols = -c("season", "competition", "game_no"),
        names_to = "player_name",
        values_to = "appearance"
    ) %>%
    filter(
        !is.na(appearance)
    ) %>%
    mutate(
        off_for = case_when(
            str_detect(appearance, "\\*") ~ "12",
            str_detect(appearance, "#") ~ "13",
            str_detect(appearance, "\\+") ~ "14",
            .default = NA_character_
        ),
        off_for = as.numeric(off_for),
        shirt_no = str_remove_all(appearance, "\\*|\\+|\\#"),
        shirt_no = as.numeric(shirt_no),
        role = ifelse(shirt_no <= 11, "starter", "sub")
    ) %>%
    arrange(
        game_no,
        shirt_no
    )

    long_ssns <- bind_rows(long_ssns, ssn_df)
}

In [47]:
final_ssn_df <- long_ssns %>%
    left_join(
        long_ssns %>%
        filter(
            !is.na(off_for) | role == "sub"
        )  %>%
        select(
            season,
            game_no,
            shirt_no,
            off_for
        ),
    by = c(
        "season" = "season",
        "game_no" = "game_no",
        "shirt_no" = "off_for")
    ) %>%
    rename(
        on_for = shirt_no.y
    ) %>%
    select(
        season,
        game_no,
        player_name,
        shirt_no,
        role,
        on_for,
        off_for
    ) %>%
    mutate(
        player_name = case_when(
            .default = player_name,
            season == "1954/55" & player_name == "MacNamara" ~ "McNamara"
        ),
        shirt_no = case_when(
            .default = shirt_no,
            season == "1997/98" & game_no == 31 & player_name == "Mahon" ~ 15,
            season == "1997/98" & game_no == 31 & player_name == "Hill" ~ 16,
            season == "1997/98" & game_no == 38 & player_name == "Mellon" ~ 15,
            season == "1998/99" & game_no == 4 & player_name == "Koumas" ~ 15,
            season == "1998/99" & game_no == 9 & player_name == "Mahon" ~ 15,
            season == "1998/99" & game_no == 11 & player_name == "Jones G" ~ 15,
            season == "1998/99" & game_no == 19 & player_name == "Challinor" ~ 15,
            season == "1998/99" & game_no == 19 & player_name == "Parkinson" ~ 16,
            season == "1998/99" & game_no == 32 & player_name == "Jones L" ~ 16
        ),
        off_for = case_when(
            .default = off_for,
            season == "1997/98" & game_no == 31 & off_for == 12 ~ 15,
            season == "1997/98" & game_no == 31 & off_for == 13 ~ 16,
            season == "1997/98" & game_no == 38 & off_for == 13 ~ 15,
            season == "1998/99" & game_no == 4 & shirt_no == 7 & player_name == "Mahon" ~ 15,
            season == "1998/99" & game_no == 9 & shirt_no == 11 & player_name == "Mellon" ~ 15,
            season == "1998/99" & game_no == 11 & shirt_no == 8 & player_name == "Santos" ~ 15,
            season == "1998/99" & game_no == 19 & shirt_no == 3 & player_name == "Thompson" ~ 15,
            season == "1998/99" & game_no == 19 & shirt_no == 7 & player_name == "Koumas" ~ 16,
            season == "1998/99" & game_no == 32 & shirt_no == 10 & player_name == "Mellon" ~ 16
        ),
        on_for = case_when(
            .default = on_for,
            season == "1998/99" & game_no == 4 & shirt_no == 15 & player_name == "Koumas" ~ 15,
            season == "1998/99" & game_no == 9 & shirt_no == 15 & player_name == "Mahon" ~ 15,
            season == "1998/99" & game_no == 11 & shirt_no == 15 & player_name == "Jones G" ~ 15,
            season == "1998/99" & game_no == 19 & shirt_no == 15 & player_name == "Challinor" ~ 15,
            season == "1998/99" & game_no == 19 & shirt_no == 16 & player_name == "Parkinson" ~ 16,
            season == "1998/99" & game_no == 32 & shirt_no == 16 & player_name == "Jones L" ~ 16
        )
    )

In [48]:
current <- read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/apps_long.csv", show_col_types = FALSE)

xls_ssns <- final_ssn_df %>%
    filter(
        !season %in% current$season
    )

In [49]:
write_csv(xls_ssns, "../output/xls_ssns.csv", na = "")    